## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
import re
import collections

# Feel free to include your text patterns functions
# from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
# Import functions to clean the script
sw = stopwords.words("english")
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#", "'"}

def remove_stop(text, sw=sw) :
    # modify this function to remove stopwords
    return([ch for ch in text if ch not in sw])

def remove_punctuation(text, punct_set=tw_punct) : 
   text = [''.join(ch for ch in word if ch not in punct_set)
           for word in text]
   
   return(text)

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    text = text.strip().split()
    return(text)

In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [4]:
# Get the table names in the database
table_names = convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

for name in table_names:
    print(name[0])

conventions


In [5]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party
                            FROM conventions;
                            ''')

for row in query_results :

    # Run the text variable through cleaning steps
    text = row[0]
    text_low = text.lower()
    text_tok = tokenize(text_low)
    text_clean = remove_punctuation(text_tok)
    text_stop = remove_stop(text_clean)
    text_untok = " ".join(text_stop)
    
    # Store party as list
    party = row[1]
    
    # Create temp list
    temp_list = [text_untok, party]
    
    # Append to convention_data
    convention_data.append(temp_list)

Let's look at some random entries and see if they look right. 

In [6]:
random.choices(convention_data,k=10)

[['mayor made lives think every housing development uncomfortable',
  'Republican'],
 ['story dan ryner he’s fifth generation farmer struggling keep business alive i’m sure president understands thinks business don’t think thinks farmers business',
  'Democratic'],
 ['china would prefer joe biden', 'Republican'],
 ['wisconsin', 'Republican'],
 ['day one john lewis role model members congress whether freshmen long time brought kind heft weightiness purpose',
  'Democratic'],
 ['arkansas', 'Republican'],
 ['illinois', 'Democratic'],
 ['he’s always eating ice cream usually it’s vanilla chocolate', 'Democratic'],
 ['kansas', 'Republican'],
 ['seen tragic shootings sides officers killing citizens citizens killing officers line duty police officers fellow citizens live country families live communities unfortunately democrats turned backs brave police officers call defunding it’s danger cities neighborhoods children isn’t ironic democrat politicians personal security protect don’t forego sec

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [7]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [8]:
def conv_features(text,fw = feature_words) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    if isinstance(text, str):
        text_tok = text.split()
    else:
        text_tok = text
    
    text_feat_words = [word for word in text_tok if word in fw]
    ret_dict = dict.fromkeys(text_feat_words, True)
    
    return ret_dict

In [9]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [10]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [11]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [12]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [13]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The classifier has found important words that help tell the difference between the Republican and Democratic parties. For example, terms like "china" and "enforcement" are strongly associated with the Republican party, while words like "votes" and "climate" are more indicative of the Democratic party. The classifier's findings align with the Democratic Party's focus on environmental issues and the Republican Party's emphasis on law enforcement and international relations. 

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [14]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [15]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [16]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for candidate, party, tweet in results:
    decoded_tweet = tweet.decode('utf-8')  # Decode the byte-string
    words = decoded_tweet.split()  # Split tweet into words
    tweet_data.append((words, party))

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [17]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [18]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: ['Earlier', 'today,', 'I', 'spoke', 'on', 'the', 'House', 'Floor', 'abt', 'protecting', 'health', 'care', 'for', 'women', 'and', 'praised', '@PPmarmonte', 'for', 'their', 'work', 'on', 'the', 'Central', 'Coast.', 'https://t.co/WqgTRzT7VV']
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: ['Go', 'Tribe!', '#RallyTogether', 'https://t.co/0NXutFL9L5']
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: ['Apparently,', 'Trump', 'thinks', "it's", 'just', 'too', 'easy', 'for', 'students', 'overwhelmed', 'by', 'the', 'crushing', 'burden', 'of', 'debt', 'to', 'pay', 'off', 'student', 'loans', '#TrumpBudget', 'https://t.co/ckYQO5T0Qh']
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: ['We’re', 'grateful', 'for', 'our', 'first', 'responders,', 'our', 'rescue', 'personnel,', 'our', 'firefighters,', 'our', 'police,', 'and', 'volunteers', 'who', 'hav

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [19]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [20]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3025, 'Democratic': 1253}),
             'Democratic': defaultdict(int,
                         {'Republican': 4087, 'Democratic': 1637})})

### Reflections

The NB classifier correctly classified 3025 tweets from the 'Republican' party and misclassified 1253 of them as 'Democratic'. On the other hand, it correctly classified 1637 tweets from the 'Democratic' party and misclassified 4087 of them as 'Republican'. There we can say that the 'Republican' party has a higher accuracy. 